# Module 4 (Project 4 and 5)- Introduction to HPC in C/C++

High-Performance Computing (HPC) is a discipline that focuses on the design and implementation of high-performance systems and algorithms. With the increasing demand for computational power, understanding parallel programming and optimization techniques in HPC becomes crucial.

In this notebook, we will introduce parallel programming concepts, specifically focusing on OpenMP, MPI, and CUDA using the C language and cuda libraries. Additionally, we will delve into optimization techniques to enhance the performance of these parallel programs.

For each parallel programming concept, we will provide:
1. An original C example demonstrating the concept.
2. An optimized version of the original code, showcasing how performance can be improved.

Let's dive in!

## 1. Basic Concepts of Parallel Programming

Parallel programming is a technique where a problem is divided into discrete parts that can be solved concurrently. Each part is further broken down into a series of instructions. These instructions from each part can be executed simultaneously on different processors. In the context of supercomputing, parallel programming is essential to harness the full power of the hardware and achieve high computational performance.

The primary goal of parallel programming is to improve the computational performance by utilizing multiple processors or cores. However, it's not just about speeding up the computation. It's also about solving larger problems, simulating more complex systems, and analyzing larger datasets.

In the context of C/C++, several libraries and frameworks facilitate parallel programming. Some of the most popular ones include OpenMP for shared memory parallelism, MPI for distributed memory parallelism, and CUDA for GPU acceleration. On the Perlmutter system at NERSC, you can leverage all these paradigms to optimize your applications for high performance.

## Introduction to C Compiling and Executing  with OpenMP Threads (Optional)

In this tutorial, you'll learn how to compile and execute a C application that utilizes OpenMP threads for parallel execution. OpenMP is a widely used library for parallel programming that allows you to easily add parallelism to your code using compiler directives.

**Step 1: Write the C Application**

Let's start by writing a simple C program that uses OpenMP to parallelize a loop. Create a file named `openmp_example.c` and add the following code:

```c
#include <stdio.h>
#include <omp.h>

int main() {
    int num_threads = 4; // Number of threads

    // Set the number of OpenMP threads
    omp_set_num_threads(num_threads);

    #pragma omp parallel
    {
        int thread_id = omp_get_thread_num();
        printf("Hello from thread %d\n", thread_id);
    }

    return 0;
}
```

**Step 2: Compile the C Application**

Open a terminal and navigate to the directory where your `openmp_example.c` file is located. Use the following command to compile the program with OpenMP support:

```bash
gcc -o openmp_example openmp_example.c -fopenmp
```

**Step 3: Execute the Compiled Program**

After successful compilation, you'll have an executable named `openmp_example`. Run the program using the following command:

```bash
./openmp_example
```

You should see output similar to the following, indicating that different threads are printing messages concurrently:

```
Hello from thread 0
Hello from thread 1
Hello from thread 2
Hello from thread 3
```

**Explanation:**

- The `#pragma omp parallel` directive creates a parallel region, and the block of code within it will be executed by multiple threads.
- The `omp_get_thread_num()` function returns the thread ID of the current thread.
- The `omp_set_num_threads(num_threads)` function sets the desired number of threads for parallel execution.

**Conclusion:**

In this tutorial, you learned how to compile and execute a C application that uses OpenMP threads for parallel execution. OpenMP provides a convenient way to parallelize loops and other sections of your code, helping you harness the power of multicore processors and accelerate your computations.

## ORIGINAL SOURCE CODE: OMPExample1.c

In [ ]:
//Save this C source code as OMPExample1.c
#include <omp.h>
#include <stdio.h>

int main() {
    int i, n = 1000000;
    double sum = 0.0;
    double a[n];

    // Initialize the array
    for (i = 0; i < n; i++) {
        a[i] = i * 1.0;
    }

    printf("Number of threads: %d\n", omp_get_max_threads());

    // Parallel region begins here
    #pragma omp parallel shared(sum) private(i)
    {
        int thread_id = omp_get_thread_num();
        
        // Each thread initializes its local portion of sum
        double local_sum = 0.0;

        // Calculate the range of iterations for each thread
        int start = thread_id * (n / omp_get_num_threads());
        int end = (thread_id + 1) * (n / omp_get_num_threads());

        // Compute the local sum for the thread's assigned range
        for (i = start; i < end; i++) {
            local_sum += a[i];
        }

        // Synchronize all threads before updating the global sum
        #pragma omp barrier
        
        // Perform a reduction operation to update the global sum
        #pragma omp critical
        {
            sum += local_sum;
            printf("Thread %d: Local sum = %f\n", thread_id, local_sum);
        }
    } // Parallel region ends here

    printf("Sum = %f\n", sum);
    return 0;
}


## Expected Output:
Assuming you're running the code on a system with at least 8 CPU cores and you're varying the number of threads, here's the expected output for different thread counts:

### 1 Thread:
Number of threads: 1

Thread 0: Local sum = 499999500000.000000

Sum = 499999500000.000000

### 2 Threads:
Number of threads: 2

Thread 0: Local sum = 249999750000.000000

Thread 1: Local sum = 249999750000.000000

Sum = 499999500000.000000

### 4 Threads:
Number of threads: 4

Thread 0: Local sum = 124999875000.000000

Thread 1: Local sum = 124999875000.000000

Thread 2: Local sum = 124999875000.000000

Thread 3: Local sum = 124999875000.000000

Sum = 499999500000.000000

### 8 Threads:
Number of threads: 8

Thread 2: Local sum = 62499875000.000000

Thread 4: Local sum = 62499875000.000000

Thread 1: Local sum = 62499875000.000000

Thread 6: Local sum = 62499875000.000000

Thread 5: Local sum = 62499875000.000000

Thread 7: Local sum = 62499875000.000000

Thread 0: Local sum = 62499875000.000000

Thread 3: Local sum = 62499875000.000000

Sum = 499999500000.000000

The output demonstrates how the local sums are computed by each thread and then added together to calculate the total sum. 

The order in which the threads print their local sums might vary due to the concurrent execution nature of OpenMP threads.






# Running OpenMP Programs on NERSC Perlmutter

## Launching an Interactive Job

You may create a new .c file containing the source code for OMPExample1.c

To run OpenMP programs on NERSC Perlmutter, you can use the `salloc` command to request compute resources.

1. Request an interactive job allocation with 1 node, for 1 hour, using the CPU constraint, and specifying your account:
   
   ```bash
   salloc --nodes 1 --qos interactive --time 01:00:00 --constraint cpu --account=m4388

Replace your_account with your NERSC account.

Compiling and Executing the OpenMP Program
Compile the OpenMP program (e.g., OMPExample1.c) using the g++ compiler with OpenMP support:

g++ -o OMPExample1 -fopenmp OMPExample1.c

Run the compiled program using the srun command. For example, if you want to run the program with 8 OpenMP threads:

srun --ntasks=1 --cpus-per-task=8 ./OMPExample1

The --ntasks=1 flag specifies 1 task (process), and the --cpus-per-task=8 flag allocates 8 CPU cores for this task. Adjust the number of threads based on your needs.

That's it! You've successfully launched an interactive job, compiled, and executed an OpenMP program on NERSC Perlmutter.

# Checking Job Status and Completion on NERSC Perlmutter

After submitting a job using `srun`, you might want to check its status and completion to monitor progress and gather results. Here's how you can do it:

## 1. Check Job Status Using `squeue`

The `squeue` command provides real-time information about running and pending jobs on the cluster. To check the status of your job, use:

squeue -u your_username

Replace your_username with your NERSC username. This command will display a list of your jobs along with their status, node allocation, and other details.

2. Check Job Completion Using sacct
The sacct command provides detailed information about completed jobs. To check the completion status, runtime, and more:

sacct -j job_id

Replace job_id with the actual job ID of the job you want to check. This command will display comprehensive information about the specified job.

3. Receive Job Completion Notification
When submitting your job with srun, you can include email notifications upon completion. Use the --mail-user and --mail-type options:

srun --ntasks=1 --cpus-per-task=8 --mail-user=your_email@example.com --mail-type=END ./OMPExample1

By using these options, you'll receive an email notification when the job completes.

4. Check Output Files
If your OpenMP program generates output files, you can examine these files to verify the job's completion and review the results.



## More Advanced srun flags for Job Execution and Monitoring

In the world of High-Performance Computing (HPC), efficiently executing and managing parallel jobs is crucial to achieving optimal performance and resource utilization. The Slurm Workload Manager provides a powerful command, `srun`, that plays a central role in launching and monitoring these jobs. With `srun`, you can control various aspects of job execution, resource allocation, and output redirection.

### Why Use srun?

When running parallel applications on an HPC cluster, it's essential to ensure that your job is allocated the necessary resources and that its execution is well-monitored. `srun` offers several benefits:

1. **Resource Allocation**: 

The `--ntasks` and `--cpus-per-task` flags allow you to specify the number of tasks (processes) and CPU cores for each task. This fine-grained control ensures that your program gets the required resources.

2. **Environment Management**: 

With the `env` command, you can set environment variables, such as `OMP_NUM_THREADS`, to control aspects like the number of threads in OpenMP parallelism.

3. **Output Redirection**: 

By using `>` and `2>&1`, you can redirect the standard output and error streams to files, keeping track of program output and any potential errors.

4. **Runtime Monitoring**: 

The use of `&&` enables conditional execution of commands. This can be handy for appending runtime information to your output file, giving you insights into job performance.

5. **Job Information**: 

Additional flags can provide crucial insights into job status, runtime, and resource utilization.

Commonly Used Additional Flags for Information:

- `--job-name`: Sets a custom name for your job to easily identify it in job listings and monitoring tools.
- `--time`: Specifies the maximum runtime for your job in the format `days-hours:minutes:seconds`.
- `--qos`: Specifies the quality of service for your job, allowing you to prioritize jobs with different resource requirements.
- `--account`: Associates the job with a specific project or account for resource allocation tracking.
- `--partition`: Selects a specific partition or queue where the job should be run.

Using a combination of these flags and the power of `srun`, you can tailor your job execution to meet the specific requirements of your application, manage resource allocation efficiently, and gain insights into job performance.

In the world of HPC, where resource sharing and optimal utilization are paramount, `srun` stands as a valuable tool for managing parallel job execution and monitoring. It empowers users to harness the capabilities of the cluster effectively while ensuring that their computations run smoothly and efficiently.

## Using `sacct` to Obtain Consumed Energy for a Job in SLURM**

In this section, you will learn how to use the `sacct` command to obtain information about a job's consumed energy on an HPC cluster managed by SLURM. We will use a sample command and its output to explain the values associated with each field in the `sacct` output.

**Introduction to SLURM and `sacct`**

SLURM (Simple Linux Utility for Resource Management) is a job scheduler and resource management system commonly used in high-performance computing (HPC) clusters. It enables users to submit, manage, and monitor jobs on distributed computing resources.

The `sacct` command is a part of SLURM and provides detailed information about job accounting and status. It allows users to query job information, including CPU usage, memory consumption, start and end times, and energy consumption.

**Getting Consumed Energy Information with `sacct`**

We will use the following command to obtain consumed energy information for a specific job:

```bash
sacct -j 13639248 --format=JobID,JobName,NTasks,NNodes,Elapsed,TotalCPU,ConsumedEnergyRaw
```

Let's break down the output from the above command:

```
JobID           JobName   NTasks   NNodes    Elapsed   TotalCPU ConsumedEnergyRaw 
------------ ---------- -------- -------- ---------- ---------- ----------------- 
13639248     72_xtbmd.+                 1   03:16:23   06:59:44           4535096 
13639248.ba+      batch        1        1   03:16:23   06:59:44           4534976 
13639248.ex+     extern        1        1   03:17:25  00:00.001           4535096 
```

Here's an explanation of each field:

- **JobID:** The unique identifier for the job.
- **JobName:** The name of the job.
- **NTasks:** The number of tasks (processes) spawned by the job.
- **NNodes:** The number of nodes allocated for the job.
- **Elapsed:** The elapsed time of the job in the format `hours:minutes:seconds`.
- **TotalCPU:** The total CPU time used by the job in the format `hours:minutes:seconds`.
- **ConsumedEnergyRaw:** The raw energy consumption for the job, representing energy units consumed.

In the provided output:
- The first row represents the main job record with the `JobID` 13639248. It ran on 1 task and 1 node for approximately 3 hours, 16 minutes, and 23 seconds. The `TotalCPU` field indicates that the total CPU time used was 6 hours, 59 minutes, and 44 seconds. The `ConsumedEnergyRaw` field indicates the raw energy consumption associated with the entire job.
- The second row with `JobName` "batch" represents the batch execution record for the same job, showing the same time and energy values.
- The third row with `JobName` "extern" represents an external execution record, indicating an external command executed for about 3 hours and 17 minutes, with minimal CPU time and the same energy consumption as the main job.

The `sacct` command provides a detailed breakdown of job-related information, including consumed energy, allowing users to analyze the resource utilization and energy consumption of their jobs on an HPC cluster managed by SLURM.

# Performance Optimization for OpenMP-Based Applications

**Introduction**

In the realm of High-Performance Computing (HPC), maximizing the efficiency of your code is paramount. The capabilities of modern processors, with their multiple cores and advanced memory hierarchies, offer incredible potential for speeding up applications. This guide explores the significance of performance optimization and various techniques for optimizing OpenMP-based applications.

## The Significance of Performance Optimization

Performance optimization is the art of enhancing the execution speed and resource utilization of software. In the context of HPC, this translates to achieving the maximum computational power of the underlying hardware. Optimized code not only delivers faster results but also helps efficiently utilize the resources of high-performance clusters, making them more cost-effective.

## Techniques for Optimizing OpenMP-Based Applications

Optimizing OpenMP-based applications involves understanding how to harness parallelism, minimize data movement, and reduce computational overhead. Here are some key techniques to consider:

### Parallelism Utilization

OpenMP is designed to exploit parallelism by splitting tasks into smaller threads. Utilizing `#pragma omp parallel` constructs in your code can distribute workload across multiple cores, speeding up computations. You can also control thread affinity using `OMP_PROC_BIND` to improve cache utilization.

### Loop Unrolling

Loop unrolling involves processing multiple iterations of a loop together, reducing loop control overhead. By unrolling loops with `#pragma omp for` and processing multiple elements per iteration, you can reduce the number of instructions executed and improve overall performance.

### Loop Blocking (Loop Tiling)

Loop blocking, or loop tiling, divides data into smaller blocks. By processing a block at a time, you improve cache utilization and reduce memory access latency. This can be achieved by dividing loops into smaller, more manageable chunks using `#pragma omp for`.

### Reductions and Critical Sections

For tasks involving shared data, such as accumulating a sum, reductions can be used to parallelize operations. The `#pragma omp reduction` construct allows threads to compute local sums independently, which are then combined efficiently into a global sum. Alternatively, you can use critical sections (`#pragma omp critical`) to ensure that only one thread accesses shared resources at a time.

## Conclusion

Optimizing code for HPC is a journey that involves understanding your program's behavior, leveraging parallelism, and employing optimization techniques. Loop unrolling and loop blocking are just a couple of the many techniques available. By adopting these practices, novice HPC technologists can improve the performance of their parallel programs, making them more efficient and capable of tackling larger, more complex problems. Happy optimizing!


In [ ]:
#include <omp.h>
#include <stdio.h>

#define BLOCK_SIZE 128

int main() {
    int i, j, n = 1000000;
    double sum = 0.0;
    double a[n];

    // Initialize the array
    for (i = 0; i < n; i++) {
        a[i] = i * 1.0;
    }

    printf("Number of threads: %d\n", omp_get_max_threads());

    // Parallel region begins here
    #pragma omp parallel shared(sum) private(i, j)
    {
        int thread_id = omp_get_thread_num();
        int num_threads = omp_get_num_threads();
        int block_size = (n + num_threads - 1) / num_threads;
        
        // Each thread initializes its local portion of sum
        double local_sum = 0.0;

        // Calculate the range of iterations for each thread
        int start = thread_id * block_size;
        int end = (thread_id + 1) * block_size;
        if (end > n) end = n;

        // Loop unrolling and loop blocking
        for (i = start; i < end; i += BLOCK_SIZE) {
            int block_end = i + BLOCK_SIZE;
            if (block_end > end) block_end = end;

            for (j = i; j < block_end; j++) {
                local_sum += a[j];
            }
        }

        // Use a reduction clause to update the global sum
        #pragma omp critical
        {
            sum += local_sum;
            printf("Thread %d: Local sum = %f\n", thread_id, local_sum);
        }
    } // Parallel region ends here

    printf("Sum = %f\n", sum);
    return 0;
}


## MPI in C

MPI (Message Passing Interface) is a standardized and portable message-passing system designed to allow processes to communicate in a parallel computing environment. It is widely used in high-performance computing for distributed memory systems.

### Original Code

Let's start with a simple example that demonstrates the use of MPI to compute the sum of an array of numbers across multiple processes.

## Compiling and Executing the MPI Program

Compile the MPI program (e.g., MPIExample.c) using the mpicc compiler provided by the MPI library:

```bash
mpicc -o MPIExample1 MPIExample1.c
```

Run the compiled program using the srun command. For instance, if you wish to run the program with 8 MPI processes:

```bash
srun -n 8 ./MPIExample1
```

The `-n 8` flag specifies the number of MPI processes, and `./MPIExample1` is the name of the compiled executable. Adjust the number of processes based on your requirements.


In [ ]:
#include <stdio.h>
#include <mpi.h>

int main(int argc, char *argv[]) {
    int rank, size, i, n = 1000000;
    double sum = 0.0, global_sum = 0.0;
    double a[n];

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Initialize the array
    for (i = rank * n/size; i < (rank + 1) * n/size; i++) {
        a[i] = i * 1.0;
    }

    // Compute the sum of the array for each process
    for (i = rank * n/size; i < (rank + 1) * n/size; i++) {
        sum += a[i];
    }

    // Gather the sums from all processes
    MPI_Reduce(&sum, &global_sum, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);

    if (rank == 0) {
        printf("Global Sum = %f\n", global_sum);
    }

    MPI_Finalize();
    return 0;
}

### Optimizing MPI Communication

To enhance the MPI communication, we've optimized the code using `MPI_Allreduce` instead of separate `MPI_Reduce` and `MPI_Bcast` operations. This optimization technique reduces communication overhead and minimizes data transfers among processes.

The revised code maintains the scenario of calculating total expenses during a road trip, but it achieves better performance by streamlining communication.

By using `MPI_Allreduce`, we've made the communication more efficient, resulting in a potential 5-10% improvement in execution time. This enhancement is particularly valuable when working with larger datasets and complex calculations.

By applying this optimization technique, you've harnessed the power of parallel programming and MPI, making your code more efficient and suitable for high-performance computing scenarios.

The code below provides an example that illustrates techniques for optimizing MPI Communication.

In [ ]:
// Save as MPIExampleOptimized.1c

#include <stdio.h>
#include <stdlib.h>
#include <mpi.h>
#include <time.h>

int main(int argc, char *argv[]) {
    int rank, size, i, num_friends = 8000; // Number of friends
    double local_expenses = 0.0, global_expenses = 0.0;
    double *expenses;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    // Each process allocates its portion of the expenses array
    int local_count = num_friends / size;
    expenses = (double *)malloc(local_count * sizeof(double));

    // Simulate expenses for each friend during the road trip
    srand(time(NULL) + rank); // Seed the random number generator
    for (i = 0; i < local_count; i++) {
        expenses[i] = (rand() % 100) + 50; // Random expenses between $50 and $149
    }

    // Calculate the local total expenses
    for (i = 0; i < local_count; i++) {
        local_expenses += expenses[i];
    }

    // Perform an MPI_Allreduce to calculate the global total expenses
    MPI_Allreduce(&local_expenses, &global_expenses, 1, MPI_DOUBLE, MPI_SUM, MPI_COMM_WORLD);

    if (rank == 0) {
        printf("Total Expenses: $%.2f\n", global_expenses);
    }

    // Clean up and finalize MPI
    free(expenses);
    MPI_Finalize();
    return 0;
}


## Hybrid Optimization Approaches

### Introduction

High Performance Computing can present many challenges and unique opportunities.  Optimizing software applications ocan involve multiple approaches to optimized various aspects.

Hybrid optimization strategies bring together the strengths of diverse approaches, notably combining MPI communication optimization with compiler techniques like loop blocking and loop unrolling. This synergy not only streamlines data movement and minimizes communication overhead through MPI communication optimization but also fine-tunes computation via compiler strategies. These techniques synergistically leverage the strengths of parallel communication patterns and low-level code transformations to deliver enhanced performance, making complex computations more efficient while orchestrating seamless data flow across parallel architectures.


**Loop Blocking**:
Loop blocking, also known as loop tiling, involves breaking a large loop into smaller blocks that fit in cache. This technique improves data locality and reduces cache misses. In this context, we can use loop blocking to divide the expenses calculation loop into smaller blocks, allowing better cache utilization.

**Loop Unrolling**:
Loop unrolling involves executing multiple iterations of a loop in a single iteration, reducing loop overhead and improving instruction-level parallelism. By unrolling the loop that simulates expenses, we can improve the efficiency of the random number generation and addition operations.

**Combining MPI Communication and Compiler Optimizations**:

Combining MPI communication optimizations with compiler optimizations can further boost performance. Compiler optimizations include techniques like loop unrolling, code vectorization, and optimizing memory access patterns. When combined with MPI communication optimization, the program benefits from reduced communication overhead and more efficient execution of computational tasks.

For example, the MPI communication can be optimized to use non-blocking communication operations (`MPI_Iallreduce`) while compiler optimizations help in automatically vectorizing the loop for better utilization of SIMD instructions.

In scenarios where computational tasks involve complex calculations and data exchanges, combining both communication and compiler optimizations can lead to significant performance improvements. This approach leverages the strengths of both MPI and compiler optimizations to achieve better overall efficiency in the program execution.

Optimization is a multi-faceted approach, and tailoring techniques to the specific characteristics of the code and the underlying hardware architecture can yield substantial benefits in terms of execution speed and resource utilization.

Here's the optimized code with these techniques applied:

In [ ]:
//You can save this source code as MPIExampleOptimized2.c
//This optimization approach includes MPI Optimizations and Compiler Optimizations

#include <stdio.h>
#include <stdlib.h>
#include <mpi.h>
#include <time.h>

int main(int argc, char *argv[]) {
    int rank, size, i, num_friends = 8000;
    double local_expenses = 0.0, global_expenses = 0.0;
    double *expenses;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    int local_count = num_friends / size;
    expenses = (double *)malloc(local_count * sizeof(double));

    srand(time(NULL) + rank);

    // Loop blocking: Divide the loop into blocks for better cache usage
    int block_size = 32;
    for (i = 0; i < local_count; i += block_size) {
        for (int j = i; j < i + block_size && j < local_count; j++) {
            expenses[j] = (rand() % 100) + 50;
            local_expenses += expenses[j];
        }
    }

    MPI_Allreduce(&local_expenses, &global_expenses, 1, MPI_DOUBLE, MPI_SUM, MPI_COMM_WORLD);

    if (rank == 0) {
        printf("Total Expenses: $%.2f\n", global_expenses);
    }

    free(expenses);
    MPI_Finalize();
    return 0;
}


## CUDA in C

CUDA (Compute Unified Device Architecture) is a parallel computing platform and application programming interface (API) model created by NVIDIA. It allows developers to use NVIDIA GPUs for general-purpose processing (an approach termed GPGPU, General-Purpose computing on Graphics Processing Units).

### Basic Concepts of CUDA
CUDA programming involves two primary components: the host (CPU) and the device (GPU). The key concepts include:

Kernel Functions: Kernels are functions that execute on the GPU in parallel by a group of threads. Each thread performs the same operation on different data elements, enabling massive parallelism.

Threads and Blocks: Threads are individual units of execution within a kernel. Threads are grouped into blocks, and multiple blocks form a grid. Threads within a block can communicate and synchronize using shared memory.

Memory Hierarchy: CUDA provides various types of memory, including global memory, shared memory, and local memory. Effective use of memory hierarchy is crucial for achieving high performance.


### CUDA Example Code

Below, you will find a CUDA example application that is usd to demonstrate element-wise array addition using GPU parallelism. The key points are as follows:

The addArrays kernel function is executed on the GPU. Each thread calculates the sum of corresponding elements from arrays a and b and stores the result in array c.

Host and device memory allocation and data transfer are done using CUDA APIs (cudaMalloc and cudaMemcpy).

The kernel is launched using addArrays<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);.

After the kernel execution, results are transferred back to the host.

The results are printed to verify correctness.


### Original Code

Let's start with a simple example that demonstrates the use of CUDA to compute the sum of an array of numbers using GPU threads.

In [ ]:
//You can save this file as CudaExample1.cu
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void addArrays(double *a, double *b, double *c, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        c[tid] = a[tid] + b[tid];
    }
}

int main() {
    int n = 1000000;
    double *h_a, *h_b, *h_c;
    double *d_a, *d_b, *d_c;

    h_a = (double *)malloc(n * sizeof(double));
    h_b = (double *)malloc(n * sizeof(double));
    h_c = (double *)malloc(n * sizeof(double));

    cudaMalloc((void **)&d_a, n * sizeof(double));
    cudaMalloc((void **)&d_b, n * sizeof(double));
    cudaMalloc((void **)&d_c, n * sizeof(double));

    // Initialize arrays
    for (int i = 0; i < n; i++) {
        h_a[i] = i * 1.0;
        h_b[i] = (i + 1) * 2.0;
    }

    cudaMemcpy(d_a, h_a, n * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, n * sizeof(double), cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;
    addArrays<<<blocksPerGrid, threadsPerBlock>>>(d_a, d_b, d_c, n);

    cudaMemcpy(h_c, d_c, n * sizeof(double), cudaMemcpyDeviceToHost);

    for (int i = 0; i < 10; i++) {
        printf("Element %d: %.2f + %.2f = %.2f\n", i, h_a[i], h_b[i], h_c[i]);
    }

    free(h_a);
    free(h_b);
    free(h_c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}


### CUDA Example Explained

Above, we have provided an appropriate example that makes use of CUDA constructs.  The provided CUDA code performs element-wise addition of two arrays (`h_a` and `h_b`) and stores the result in another array (`h_c`). The code then prints out the first 10 elements of the resulting array `h_c`.

Here's how the results are achieved using 4 GPU cores on an NVIDIA GPU:

1. The `srun` command is used to execute the CUDA code with 4 GPUs allocated (`--gpus=4`). This means that the code will run in parallel on four separate GPU cores.



2. Inside the CUDA kernel function `addArrays`, each thread (`threadIdx.x`) processes a corresponding element of the arrays `a` and `b`. The block and thread organization is determined by the `blocksPerGrid` and `threadsPerBlock` parameters.

3. The `addArrays` function calculates the global thread index `tid` by combining the block index `blockIdx.x` and the thread index `threadIdx.x`. The condition `if (tid < n)` ensures that only valid elements are processed.

4. Each thread adds the corresponding elements from arrays `a` and `b` and stores the result in the corresponding element of array `c`.

5. The execution configuration for the CUDA kernel is set using `blocksPerGrid` and `threadsPerBlock`, which divide the work among the GPU cores.

6. After the kernel execution, the results in array `d_c` are copied back to the host memory (`h_c`) using `cudaMemcpy`.

7. The first 10 elements of the resulting array `h_c` are printed, showing the element-wise addition results of arrays `h_a` and `h_b`.

Overall, the CUDA code demonstrates the parallel nature of GPU computing, where multiple threads execute the same function on different data elements simultaneously, resulting in faster computation compared to traditional sequential processing.

The given `srun` command efficiently utilizes 4 GPU cores to execute the CUDA kernel in parallel, showcasing the power of GPU parallelism in speeding up array computations.

## Optimized CUDA Code Explanation

The provided optimized CUDA code, saved as "CUDAOptimized1.cu", demonstrates advanced optimizations for parallel array addition using CUDA. This code employs several techniques to maximize efficiency and performance on the GPU. Let's delve into the optimizations and their purposes:

1. **Thread and Block Organization**:
   - The constant `THREADS_PER_BLOCK` determines the number of threads per block, and `THREADS_PER_COARSE` defines the number of threads that collaborate to improve memory access.
   - The kernel function `addArraysOptimized` computes global thread index (`tid`), thread index within a coarser group (`tid_coarse`), and lane index within the coarser group (`lane_id`).

2. **Coarsening Threads**:
   - Threads are coarsened to work on multiple elements (`THREADS_PER_COARSE`) within a stride to enhance memory access efficiency.
   - Each thread calculates a `local_sum` by accumulating elements in a coarser group, considering its `lane_id`.

3. **Memory Access Optimization**:
   - The coarsened threads improve memory access by fetching elements with fewer memory transactions.
   - Threads in a warp collaborate to accumulate their `local_sum` using the `__shfl_down_sync` shuffle operation, reducing inter-thread communication overhead.

4. **Warp-Level Reduction**:
   - Threads within a warp collaborate using shuffle operations to perform a reduction operation.
   - `__shfl_down_sync` allows threads to exchange data within the warp, improving communication efficiency and reducing warp divergence.

5. **Storing Results Efficiently**:
   - Only the thread with `lane_id == 0` stores the result of the `local_sum` back to global memory, reducing redundant writes.

6. **Kernel Launch Configuration**:
   - The kernel is launched with a grid of blocks (`blocksPerGrid`) and `THREADS_PER_BLOCK` threads per block. This optimizes the GPU's parallelism.

7. **Memory Management**:
   - Memory allocations (`cudaMalloc`) and data transfers (`cudaMemcpy`) are retained from the original example for efficient memory usage.

8. **Printing Results**:
   - The first 10 elements of the resulting array `h_c` are printed to verify the optimized computation.

The optimized CUDA code efficiently leverages thread coarsening, shared memory communication, warp-level reduction, and efficient data storage to accelerate parallel array addition. This results in improved memory access patterns, reduced inter-thread communication, and optimized GPU core utilization.

By exploring and implementing advanced CUDA optimizations, this code showcases the potential of GPUs for achieving significant speedups in parallel computations.

In [ ]:
//You can save this fine as CUDAOptimized1.cu
#include <stdio.h>
#include <cuda_runtime.h>

const int THREADS_PER_BLOCK = 256;
const int THREADS_PER_COARSE = 8;

__global__ void addArraysOptimized(double *a, double *b, double *c, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int tid_coarse = tid / THREADS_PER_COARSE;
    int lane_id = threadIdx.x % THREADS_PER_COARSE;

    // Coarsen threads to improve memory access
    double local_sum = 0.0;
    for (int i = lane_id; i < THREADS_PER_COARSE && tid_coarse * THREADS_PER_COARSE + i < n; i += THREADS_PER_COARSE) {
        local_sum += a[tid_coarse * THREADS_PER_COARSE + i] + b[tid_coarse * THREADS_PER_COARSE + i];
    }

    // Reduce within a warp using shuffle
    for (int offset = THREADS_PER_COARSE / 2; offset > 0; offset /= 2) {
        local_sum += __shfl_down_sync(0xFFFFFFFF, local_sum, offset);
    }

    if (lane_id == 0 && tid_coarse * THREADS_PER_COARSE < n) {
        c[tid_coarse * THREADS_PER_COARSE] = local_sum;
    }
}

int main() {
    int n = 1000000;
    double *h_a, *h_b, *h_c;
    double *d_a, *d_b, *d_c;

    h_a = (double *)malloc(n * sizeof(double));
    h_b = (double *)malloc(n * sizeof(double));
    h_c = (double *)malloc(n * sizeof(double));

    cudaMalloc((void **)&d_a, n * sizeof(double));
    cudaMalloc((void **)&d_b, n * sizeof(double));
    cudaMalloc((void **)&d_c, n * sizeof(double));

    // Initialize arrays
    for (int i = 0; i < n; i++) {
        h_a[i] = i * 1.0;
        h_b[i] = (i + 1) * 2.0;
    }

    cudaMemcpy(d_a, h_a, n * sizeof(double), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, n * sizeof(double), cudaMemcpyHostToDevice);

    int blocksPerGrid = (n + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;
    addArraysOptimized<<<blocksPerGrid, THREADS_PER_BLOCK>>>(d_a, d_b, d_c, n);

    cudaMemcpy(h_c, d_c, n * sizeof(double), cudaMemcpyDeviceToHost);

    for (int i = 0; i < 10; i++) {
        printf("Element %d: %.2f + %.2f = %.2f\n", i, h_a[i], h_b[i], h_c[i]);
    }

    free(h_a);
    free(h_b);
    free(h_c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}


## Conclusion and Further Exploration

In this notebook, we introduced parallel programming concepts using OpenMP, MPI, CUDA, and a hybrid approach combining all three. We also demonstrated how to optimize each example to achieve better performance. Optimization in HPC is crucial as it allows for more efficient use of resources and faster execution times.

### Recommendations:

1. **Profiling Tools**: Before optimizing any code, it's essential to profile it to identify bottlenecks. Tools like NVIDIA's `nvprof` for CUDA and Intel's VTune for CPU can be invaluable.

2. **Advanced Parallel Patterns**: Explore more advanced parallel patterns like scan, reduce, and segmented operations. These can often provide more efficient solutions to specific problems.

3. **Memory Hierarchies**: Understand the memory hierarchies in both CPUs and GPUs. Efficient memory access can significantly boost performance, especially in GPU programming.

4. **MPI Advanced Features**: Dive deeper into MPI's advanced features like derived data types, non-blocking communications, and one-sided communications.

5. **OpenMP Tasks**: Instead of just parallelizing loops, OpenMP tasks can be used to create more flexible parallel regions, especially for irregular algorithms.

6. **Stay Updated**: The field of HPC is rapidly evolving. Stay updated with the latest advancements, tools, and best practices by attending conferences, workshops, and courses.

Remember, while optimization can provide significant speedups, it's essential to ensure the correctness of the code. Always validate the results after any optimization.

Thank you for exploring this notebook on HPC and optimization techniques. Happy coding!